In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import pandas as pd
import re
import csv
import numpy as np
import math
import random

In [3]:
print(tf.__version__)

2.4.1


In [4]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [3]:
train_all = pd.read_csv("all2.csv", delimiter = ",")
train_all.isnull().values.any()
train_all.shape

(6420, 3)

In [43]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [44]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [52]:
articles = []
labels = []

with open("all2.csv", 'r', encoding = 'utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[2])
        #if(len(row)>13):
            #article = row[2]+row[3]+row[4]+row[5]+row[7]+row[13]
        #else:
        article = row[1]
        articles.append(preprocess_text(article))

print(len(articles))

6420


In [53]:
train_all.label.unique()

array(['real', 'fake'], dtype=object)

In [54]:
def tokenize_articles(txt):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(txt))

In [55]:
tokenized_articles = [tokenize_articles(article) for article in articles]
print(len(tokenized_articles))
print(tokenized_articles[7])

6420
[8112, 4455, 8398, 21887, 23350, 3433, 19633, 7071, 16770, 2522, 2139, 2094, 4160, 4371, 14949, 2497]


In [57]:
y = train_all['label']
y = np.array(list(map(lambda x: 1 if x=='real' else 0, y)))

print(y)

[1 1 0 ... 0 0 1]


In [58]:
print(tokenized_articles)
input_with_len = [[article, y[i], len(article)]
            for i, article in enumerate(tokenized_articles)]

[[1996, 26629, 2747, 4311, 6677, 1999, 2236, 1996, 5860, 2890, 9739, 9243, 1999, 2331, 9294, 2090, 2367, 4216, 2024, 2235, 1998, 4654, 24759, 5555, 3468, 1996, 2331, 9565, 4832, 2012, 5560, 2111, 2651], [2163, 2988, 6677, 2235, 4125, 2013, 2197, 9857, 2670, 2163, 2988, 1997, 2216, 6677, 16770, 2522, 8038, 28745, 5339, 2102, 1057, 2595], [10317, 6149, 2450, 2471, 3594, 6090, 3207, 7712, 2004, 8016, 2025, 2000, 2128, 8557, 6081, 4524, 16770, 2522, 16215, 2546, 3282, 22540, 2063, 21887, 23350, 8423], [2634, 20450, 9363, 26788, 2057, 2031, 2522, 17258, 5604, 12030, 1999, 2634, 1998, 2004, 2006, 16215, 2257, 5852, 2031, 2042, 2589, 11268, 22655, 28921, 3567, 1040, 2290, 24582, 2213, 25547, 2140, 4048, 12237, 10354, 2063, 2634, 29602, 2140, 10105, 16770, 2522, 1061, 2232, 1062, 2595, 2243, 10695, 2232, 2480], [20151, 2163, 2064, 9699, 2312, 2553, 9294, 2021, 2065, 2017, 2298, 2012, 1996, 2047, 3572, 2566, 2454, 2651, 3760, 2163, 2024, 4760, 2062, 3572, 2566, 2454, 2084, 2662, 2030, 3146, 263

In [59]:
random.shuffle(input_with_len)
for i in range(0, 9999):
    print(input_with_len[i][1])

1
1
1
0
1
0
1
1
1
1
0
0
1
1
0
0
1
1
0
0
1
1
0
0
1
1
1
0
0
0
0
0
1
0
0
1
0
0
1
1
1
0
0
1
1
1
0
0
0
1
0
1
1
0
1
0
1
0
0
0
0
0
1
1
0
0
0
1
0
1
1
0
1
0
0
1
1
0
0
0
1
1
0
1
1
1
1
1
1
1
1
0
1
0
0
0
1
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
0
1
1
0
0
0
1
0
1
0
1
0
0
0
1
1
1
1
1
0
1
1
0
1
1
1
0
1
1
0
1
0
1
0
0
1
1
0
0
0
0
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
1
0
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
1
1
0
1
0
0
0
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
0
0
1
1
1
1
0
0
0
1
1
1
1
1
0
1
1
1
1
0
0
0
1
0
1
1
0
1
0
1
0
1
0
0
1
1
1
1
0
1
1
0
1
0
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
0
1
0
1
0
1
1
1
0
1
1
1
0
1
0
1
0
0
1
1
1
0
0
1
0
1
1
1
0
1
0
1
0
1
0
1
1
1
1
0
1
1
0
1
1
1
1
0
1
0
1
1
1
0
0
1
1
1
0
1
1
1
0
1
1
1
0
1
0
0
1
1
1
1
0
0
1
0
1
1
1
1
0
0
1
0
0
1
1
1
0
1
1
1
1
0
1
1
0
0
0
0
0
1
0
0
0
1
1
1
0
1
1
0
0
1
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
1
1
1
0
1
0
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
1
0
1
1
0
0
0
1
1
1
0
0
0
0
1
0
0
1
0
0
1
0
0
1
0
0
0
1
0
1
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
0
0
0
1
1
1
0
0
1


1
1
1
0
1
1
1
1
0
0
0
1
0
1
0
1
1
1
1
1
0
0
0
1
1
1
1
1
0
1
0
1
0
0
1
1
1
1
1
1
0
0
0
1
1
0
1
1
1
0
1
1
0
1
1
0
1
1
0
1
1
1
1
1
0
1
1
0
1
0
0
0
1
1
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
1
1
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
0
0
1
0
0
0
0
0
1
0
0
1
0
1
1
0
1
0
1
0
1
0
0
0
1
0
0
1
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
0
0
1
1
1
1
0
0
1
0
0
1
0
1
0
1
0
1
1
1
1
0
1
0
1
1
1
0
1
1
0
1
1
0
1
0
1
1
1
0
0
0
0
1
1
0
1
0
0
1
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
1
1
1
0
0
0
0
0
0
1
1
1
1
0
1
1
1
0
0
1
1
1
1
0
0
1
1
1
1
1
1
1
0
0
1
0
0
0
1
1
0
1
0
1
1
1
1
1
1
0
0
1
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
0
1
0
1
1
1
1
0
0
0
1
1
0
0
0
1
0
1
0
1
1
0
0
1
0
0
1
0
1
0
1
1
0
0
1
0
0
0
1
1
1
1
0
1
0
1
1
0
1
0
0
0
1
0
0
1
0
0
0
0
1
0
1
0
0
1
0
0
0
1
0
1
1
0
0
1
0
1
1
0
0
0
1
0
1
0
0
0
1
0
1
0
1
0
0
0
1
1
1
0
0
1
0
0
1
1
1
1
0
0
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
0
1
1
1
0
1
0
1
1
0
1
0
0
1
0
0
1
0
0
0
0
1
1
1
0
0
0
1
1
1
1
0
0
1
0
1
1
0
1
1
0
0
1
0
0
1
0
1
0
1
0
1
0
0
1
0
1
1
0
0
0


IndexError: list index out of range

In [60]:
input_with_len.sort(key=lambda x: x[2])

In [61]:
sorted_articles_labels = [(article_lab[0], article_lab[1]) for article_lab in input_with_len]
for i in range(0, 9999):
    print(sorted_articles_labels[i][1])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
1
0
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
0
1
0
1
1
0
0
1
0
0
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
0
0
0
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
1
0
0
0
0
0
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
0
0
0
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
0
1
1
1
0
0
0
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
0
1
0
1
1
1
0
1
1
1
0
1
1
1
0
1
1
1
1
1
1
0
1
1
1
0
1
1
1
0
0
1
0
1
1
1
0
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1


IndexError: list index out of range

In [62]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_articles_labels, output_types=(tf.int32, tf.int32))

In [63]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 7), dtype=int32, numpy=
 array([[ 2522, 17258,  2003, 10519,     0,     0,     0],
        [ 2522, 17258,  3231,  9861,     0,     0,     0],
        [21887, 23350,  7207,  2015,     0,     0,     0],
        [16034,  8563, 21887, 23350,     0,     0,     0],
        [10356, 15153,  2022, 22381,  3490,     0,     0],
        [16034,  9526,  2015,  2522, 17258,     0,     0],
        [ 2040,  2904,  2049, 16449,  2015,     0,     0],
        [14906, 15403,  2038,  2042,  2584,     0,     0],
        [ 2053,  2062, 21887,  7865,  2331,     0,     0],
        [17901,  8563,  1996, 21887, 23350,     0,     0],
        [ 4157,  9526,  2015, 21887, 23350,     0,     0],
        [ 5948,  6544,  2064,  9526,  2522, 17258,     0],
        [17404,  2114, 21887, 23350,  2001,  2764,     0],
        [11884,  2015,  2024,  5255,  2822,  5324,     0],
        [ 4147, 15806,  2038,  2042, 10003, 20694,     0],
        [21887, 23350,  2003,  3303,  2011, 12971,     0],
        

In [64]:
TOTAL_BATCHES = math.ceil(len(sorted_articles_labels) / BATCH_SIZE)
print(TOTAL_BATCHES)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

201


In [65]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [66]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 1

In [67]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [68]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [69]:
text_model.fit(train_data, epochs=NB_EPOCHS)

181/181 [==============================] - 28s 140ms/step - loss: 0.4929 - accuracy: 0.7619


In [72]:
results = text_model.evaluate(test_data)
print(results)

31/31 [==============================] - 1s 22ms/step - loss: 1.1306e-04 - accuracy: 1.0000
[0.00011305962834740058, 1.0]


In [31]:
text_model.summary()

Model: "text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  6104400   
_________________________________________________________________
conv1d_6 (Conv1D)            multiple                  40100     
_________________________________________________________________
conv1d_7 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_8 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d_2 (Glob multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  77056     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0

In [4]:
from collections import Counter
print(Counter(train_all['label'].values))

Counter({'real': 3360, 'fake': 3060})
